In [15]:
import os
import boto3
import sagemaker
import numpy as np
from sagemaker.tensorflow import TensorFlowModel
from sagemaker.tensorflow import TensorFlowPredictor
from keras.preprocessing.image import load_img, img_to_array

In [2]:
session = boto3.Session(
    aws_access_key_id='XXXXXXXXXXXXXXXXXXXXXXXXXX',
    aws_secret_access_key='XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX',
    region_name='XXXXXXXXXX'
)

In [3]:
model_data = "s3://my-image-classification/models/final.tar.gz"

In [4]:
def deploy_model(s3_model_path, sagemaker_role):
    model = TensorFlowModel(model_data=s3_model_path, role=sagemaker_role, framework_version="2.4.1")
    predictor = model.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")
    
    return predictor

In [17]:
def claasify(image_path):
    class_map = {0: 'human', 1: 'no_human', 2: 'uber_eats'}

    endpoint = 'tensorflow-inference-2024-02-04-05-46-29-688'
    predictor = TensorFlowPredictor(endpoint)

    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)

    # Preprocess the image
    img_batch = np.expand_dims(img_array, axis=0)
    img_batch = img_batch / 255.0  # Normalize to [0, 1] range

    inference_response = predictor.predict(img_batch)
    predictions = np.array(inference_response['predictions'][0])

    pred_class = predictions.argmax()
    pred_prob = predictions[pred_class]

    pred_label = class_map[pred_class]

    return pred_label

In [6]:
deploy_model(model_data, "sagemaker-role")

---!

In [18]:
test_path = "/Users/deveshparmar/Citadel/Assignments/Hawells/split_dataset_2/test"
image_path = os.path.join(test_path, "human", "434.png")
label = claasify(image_path)
print(label)

human


In [19]:
import os
import json

## Convert to TF servable model

In [5]:
my_model = load_model('tl_final_model_v1.weights.best.hdf5')

In [7]:
model_save_path = os.path.join(os.getcwd(), "final_model/1")

In [8]:
tf.saved_model.save(my_model, model_save_path)

INFO:tensorflow:Assets written to: /Users/deveshparmar/Citadel/Assignments/Hawells/final_model/1/assets


INFO:tensorflow:Assets written to: /Users/deveshparmar/Citadel/Assignments/Hawells/final_model/1/assets
